# 总日志分析
## 1. 总日志txt特点
- `bugreport-`开头
- 大约两百万行日志
- 包含唤醒方案等相关文字信息
- 包含其他系统信息
## 2. 需求分析
1. 可以得到唤醒方案、算法、机型等数据；
2. 可以分析每一行日志，但是时间比较长；
3. 可以筛选出某一时刻在某一时间内的全部有效日志；
    - 全部有效日志是指包含voicetrigger项目中关键log打印的关键词；
    - 全部关键词放在一个json文件中；
    - 难点：
        **如何分析和展示某一时刻附近发生的问题？**
        1. 快速获得有效时间内的全部日志（前后7分钟内日志大约30万行）
        1. 分析该时刻前后7分钟的全部日志，得出附近的一级问题和二级问题；
        2. 交给用户来确定分析该时刻的附近几分钟（也可以考虑）；
## 3. 难点
1. ~二分法快速锁定某一范围的日志（筛选的日志有遗漏，需要其他优化方法）~
    1. 正则匹配确定时间
        OK
    2. 二分法确定范围
        OK
    3. 正则匹配 + 有效日志关键词筛选得到最终有效日志
        OK

2. 对于不是完全按照时间顺序的日志，如何快速筛选一段时间内的有效日志
    **速度上可以继续优化**
    1. 直接每次分析每2000行的第一条和最后一条日志的时间。如果其中一条日志满足，就直接将这1000行日志最为有效日志存起来；
    2. 对于遍历大数量的日志确实能提高效率，比逐行遍历每一条日志提高效率：
        1. 逐行遍历时间复杂度为`O(n)`
        2. 间隔2000行粗略筛选，时间复杂度为`O(n/2000)`
        3. 为什么是选择间隔1000行。因为我发现一段有效且连续在一分钟内的日志一定大于2000行。
    3. 粗略筛选之后还需要进行第二次细致筛选:
        正则匹配 + 有效日志关键词筛选得到最终有效日志


## 4. 日志问题关键词匹配
1. 得到唤醒方案、机型等文字信息
    TODO:貌似需要分析全部日志，个人认为可以通过后续在导出的表格中直接获取，没必要浪费时间分析全部日志
    1. voice_trigger_version
    关键日志：
    ```
    # voice_trigger_version
    Package [com.miui.voicetrigger] (8c57a04):
    appId=10181
    pkg=Package{a5a5bed com.miui.voicetrigger}
    codePath=/product/app/VoiceTrigger
    resourcePath=/product/app/VoiceTrigger
    legacyNativeLibraryDir=/product/app/VoiceTrigger/lib
    extractNativeLibs=true
    primaryCpuAbi=arm64-v8a
    secondaryCpuAbi=null
    cpuAbiOverride=null
    versionCode=2024060310 minSdk=26 targetSdk=33
    minExtensionVersions=[]
    versionName=v-5.9.0.11-qcom
    ```
    关键代码
    ```python
    if not is_get_package_info and re.search('Package \[com.miui.voicetrigger\]', self.log_lines[i]) is not None:
        self.build_date = self.log_lines[i+10].split('=')[1]
        self.voice_trigger_version = self.log_lines[i+12].split('=')[1:]
        is_get_package_info = True
    ```


2. 未收到一级唤醒事件
四种方案通用
未查询到关键日志点‘onRecognition:’，则标记为：未接收到一级事件
```bash
(高通/自研E) D/SVA-LegacyWakeupSession - onRecognition: recognitionEvent = KeyphraseRecognitionEvent [keyphraseExtras=[KeyphraseRecognitionExtra [id=0, recognitionModes=3, coarseConfidenceLevel=98, confidenceLevels=[ConfidenceLevel [userId=1, confidenceLevel=14]]]], status=0, soundModelHandle=1, captureAvailable=true, captureSession=25, captureDelayMs=0, capturePreambleMs=0, triggerInData=false, sampleRate=16000, encoding=2, channelMask=16, data=8944]
(MTK) V/VoiceTriggerManager - onRecognition: GenericRecognitionEvent ::RecognitionEvent [status=0, soundModelHandle=-1, captureAvailable=false, captureSession=-1, captureDelayMs=-1, capturePreambleMs=-1, triggerInData=false, sampleRate=16000, encoding=2, channelMask=1, data=0]
(MTK自研) V/MTKVoiceTriggerManager - onRecognition: GenericRecognitionEvent ::RecognitionEvent [status=0, soundModelHandle=-1, captureAvailable=false, captureSession=297, captureDelayMs=8704, capturePreambleMs=-1, triggerInData=false, sampleRate=16000, encoding=2, channelMask=1, data=0] false
```

3. 一级唤醒事件



4. 二级唤醒事件



## 难点分析之：获取某一时刻在某一范围内的关键日志
当然可以，下面是对代码中涉及的关键知识点的详细讲解：

### 1. 正则表达式匹配时间戳

```python
timestamp_pattern = re.compile(r'(\d{2}-\d{2} \d{2}:\d{2}:\d{2})')
```

- **正则表达式**：用于模式匹配字符串，这里使用 `\d{2}-\d{2} \d{2}:\d{2}:\d{2}` 模式来匹配形如 "MM-DD HH:MM:SS" 的时间戳。
  - `\d{2}`：匹配两位数字。
  - `-` 和 `:`：匹配特定的分隔符。
  - 空格：匹配日期和时间之间的空格。
- **re.compile**：将正则表达式编译成一个模式对象，以提高匹配效率。

### 2. 解析时间戳

```python
def parse_log_timestamp(line):
    match = timestamp_pattern.search(line)
    if match:
        timestamp_str = match.group(1)
        return datetime.datetime.strptime(timestamp_str, '%m-%d %H:%M:%S').replace(year=datetime.datetime.now().year)
    return None
```

- **re.search**：在字符串中查找正则表达式模式的第一个匹配项。
- **match.group(1)**：获取匹配的第一个子组（即时间戳）。
- **datetime.datetime.strptime**：将字符串解析为 `datetime` 对象。
- **replace(year=...)**：因为日志中没有年份，默认使用当前年份。

### 3. 二分查找

```python
def find_log_index(lines, target_time_str):
    target_time = datetime.datetime.strptime(target_time_str, '%m-%d %H:%M:%S').replace(year=datetime.datetime.now().year)
    
    low, high = 0, len(lines) - 1
    
    while low <= high:
        mid = (low + high) // 2
        mid_time = parse_log_timestamp(lines[mid])
        
        if mid_time is None:
            high = mid - 1
            continue
        
        if mid_time < target_time:
            low = mid + 1
        elif mid_time > target_time:
            high = mid - 1
        else:
            return mid
    
    return low
```

- **二分查找**：一种高效的查找算法，适用于已经排序的数组或列表。
  - 通过不断将搜索范围减半，快速定位目标元素的位置。
- **逻辑**：
  - 初始化 `low` 和 `high` 指针。
  - 在 `low <= high` 条件下进行循环：
    - 计算中间位置 `mid`。
    - 解析中间位置的时间戳 `mid_time`。
    - 根据 `mid_time` 与 `target_time` 的比较结果，调整 `low` 或 `high` 指针。

### 4. 提取目标时间范围的日志

```python
def extract_logs(filename, target_time_str, delta_minutes=3):
    with open(filename, 'r', encoding="utf-8") as file:
        lines = file.readlines()
    
    target_index = find_log_index(lines, target_time_str)
    
    target_time = datetime.datetime.strptime(target_time_str, '%m-%d %H:%M:%S').replace(year=datetime.datetime.now().year)
    delta = datetime.timedelta(minutes=delta_minutes)
    
    start_time = target_time - delta
    end_time = target_time + delta

    start_index = find_log_index(lines, start_time.strftime('%m-%d %H:%M:%S'))
    end_index = find_log_index(lines, end_time.strftime('%m-%d %H:%M:%S'))
    
    start_index = max(0, start_index - 1)
    end_index = min(len(lines) - 1, end_index + 1)
    
    return lines[start_index:end_index + 1]
```

- **读取文件内容**：
  - `with open(filename, 'r', encoding="utf-8") as file`：打开文件并读取所有行。
- **目标时间和时间范围**：
  - `datetime.datetime.strptime`：解析目标时间字符串为 `datetime` 对象。
  - `datetime.timedelta`：创建一个时间间隔对象。
  - 计算目标时间前后 `delta_minutes` 分钟的时间范围。
- **二分查找**：
  - 使用 `find_log_index` 找到目标时间及其前后时间范围的索引。
- **处理边界情况**：
  - 确保 `start_index` 和 `end_index` 在合法范围内。
- **返回提取的日志行**：
  - 返回目标时间范围内的日志行。



In [11]:
import re
import datetime

# 正则表达式用于匹配时间戳，不包括毫秒部分
timestamp_pattern = re.compile(r'(\d{2}-\d{2} \d{2}:\d{2}:\d{2})')

def parse_log_timestamp(line):
    match = timestamp_pattern.search(line)
    if match:
        timestamp_str = match.group(1)
        return datetime.datetime.strptime(timestamp_str, '%m-%d %H:%M:%S').replace(year=datetime.datetime.now().year)
    return None

# 二分判断
def find_log_index(lines, target_time_str):
    target_time = datetime.datetime.strptime(target_time_str, '%m-%d %H:%M:%S').replace(year=datetime.datetime.now().year)
    
    low, high = 0, len(lines) - 1
    
    while low <= high:
        mid = (low + high) // 2
        mid_time = parse_log_timestamp(lines[mid])
        
        if mid_time is None:
            high = mid - 1
            continue
        
        if mid_time < target_time:
            low = mid + 1
        elif mid_time > target_time:
            high = mid - 1
        else:
            return mid
    
    return low

# 二分查找确定关键日志
def extract_logs(filename, target_time_str, pre_delta_minutes=3, post_delta_minutes=3):
    with open(filename, 'r', encoding="utf-8") as file:
        lines = file.readlines()
    
    
    target_time = datetime.datetime.strptime(target_time_str, '%m-%d %H:%M:%S').replace(year=datetime.datetime.now().year)
    
    # 前3分钟
    pre_delta = datetime.timedelta(minutes=pre_delta_minutes)
    # 后2分钟
    post_delta = datetime.timedelta(minutes=post_delta_minutes)
    
    start_time = target_time - pre_delta
    end_time = target_time + post_delta

    start_index = find_log_index(lines, start_time.strftime('%m-%d %H:%M:%S'))
    end_index = find_log_index(lines, end_time.strftime('%m-%d %H:%M:%S'))
    
    # 处理找不到确切位置的情况，确保 start_index 和 end_index 合法
    start_index = max(0, start_index - 1)
    end_index = min(len(lines) - 1, end_index + 1)
    
    # 提取目标日志范围
    return lines[start_index:end_index + 1]

# 示例使用
log_filename = 'bugreport-goku-UKQ1.240116.001-2024-07-06-22-06-59.txt'
target_time_str = '07-06 22:04:40'
pre_delta_minutes = 3
post_delta_minutes = 2

logs = extract_logs(log_filename, target_time_str, pre_delta_minutes, post_delta_minutes)

# 将提取的日志输出到一个新的文件
with open('output.txt', 'w', encoding="utf-8") as output_file:
    output_file.writelines(logs)


## 难点分析之：有效日志关键词筛选
包含voicetrigger项目的关键词在keyword.json文件中。规定时间内的日志再次进行筛选。
1. 读取 JSON 文件
2. 筛选日志中包含关键词的行
3. 输出筛选后的日志

In [4]:
import re
import datetime
import json

# 正则表达式用于匹配时间戳，不包括毫秒部分
timestamp_pattern = re.compile(r'(\d{2}-\d{2} \d{2}:\d{2}:\d{2})')

def parse_log_timestamp(line):
    match = timestamp_pattern.search(line)
    if match:
        timestamp_str = match.group(1)
        return datetime.datetime.strptime(timestamp_str, '%m-%d %H:%M:%S').replace(year=datetime.datetime.now().year)
    return None

# 二分判断
def find_log_index(lines, target_time_str):
    target_time = datetime.datetime.strptime(target_time_str, '%m-%d %H:%M:%S').replace(year=datetime.datetime.now().year)
    
    low, high = 0, len(lines) - 1
    
    while low <= high:
        mid = (low + high) // 2
        mid_time = parse_log_timestamp(lines[mid])
        
        if mid_time is None:
            high = mid - 1
            continue
        
        if mid_time < target_time:
            low = mid + 1
        elif mid_time > target_time:
            high = mid - 1
        else:
            return mid
    
    return low

# 二分查找确定关键日志
def extract_logs(filename, target_time_str, pre_delta_minutes=3, post_delta_minutes=2):
    with open(filename, 'r', encoding="utf-8") as file:
        lines = file.readlines()
    
    target_index = find_log_index(lines, target_time_str)
    
    target_time = datetime.datetime.strptime(target_time_str, '%m-%d %H:%M:%S').replace(year=datetime.datetime.now().year)
    
    # 前3分钟
    pre_delta = datetime.timedelta(minutes=pre_delta_minutes)
    # 后2分钟
    post_delta = datetime.timedelta(minutes=post_delta_minutes)
    
    start_time = target_time - pre_delta
    end_time = target_time + post_delta

    start_index = find_log_index(lines, start_time.strftime('%m-%d %H:%M:%S'))
    end_index = find_log_index(lines, end_time.strftime('%m-%d %H:%M:%S'))
    
    # 处理找不到确切位置的情况，确保 start_index 和 end_index 合法
    start_index = max(0, start_index - 1)
    end_index = min(len(lines) - 1, end_index + 1)
    
    # 提取目标日志范围
    return lines[start_index:end_index + 1]

# 筛选包含关键词的日志
def filter_logs_by_keywords(logs, keywords):
    filtered_logs = []
    for line in logs:
        if any(keyword in line for keyword in keywords):
            filtered_logs.append(line)
    return filtered_logs

# 示例使用
log_filename = 'bugreport-goku-UKQ1.240116.001-2024-07-06-22-06-59.txt'
target_time_str = '07-06 20:29:45'
pre_delta_minutes = 3
post_delta_minutes = 2

# 提取关键时间范围内的日志
logs = extract_logs(log_filename, target_time_str, pre_delta_minutes, post_delta_minutes)

# 读取关键词JSON文件
with open('voicetrigger_keywords.json', 'r', encoding="utf-8") as file:
    keywords_data = json.load(file)
keywords = keywords_data['keywords']

# 筛选包含关键词的日志
filtered_logs = filter_logs_by_keywords(logs, keywords)

# 将筛选后的日志输出到一个新的文件
with open('filtered_output.txt', 'w', encoding="utf-8") as output_file:
    output_file.writelines(filtered_logs)


## 分析有误，不能按照二分的方式去分析
所以先按照全部遍历的方式，然后去优化

In [17]:
import re
import datetime

# 正则表达式用于匹配时间戳，不包括毫秒部分
timestamp_pattern = re.compile(r'(\d{2}-\d{2} \d{2}:\d{2}:\d{2})')

def parse_log_timestamp(line):
    match = timestamp_pattern.search(line)
    if match:
        timestamp_str = match.group(1)
        return datetime.datetime.strptime(timestamp_str, '%m-%d %H:%M:%S').replace(year=datetime.datetime.now().year)
    return None

def extract_logs(filename, target_time_str, pre_delta_minutes=3, post_delta_minutes=2):
    with open(filename, 'r', encoding="utf-8") as file:
        lines = file.readlines()

    target_time = datetime.datetime.strptime(target_time_str, '%m-%d %H:%M:%S').replace(year=datetime.datetime.now().year)
    
    # 前3分钟
    pre_delta = datetime.timedelta(minutes=pre_delta_minutes)
    # 后2分钟
    post_delta = datetime.timedelta(minutes=post_delta_minutes)
    
    start_time = target_time - pre_delta
    end_time = target_time + post_delta

    result_lines = []
    within_time_range = False

    for line in lines:
        line_time = parse_log_timestamp(line)
        
        if line_time:
            within_time_range = start_time <= line_time <= end_time
            if within_time_range:
                result_lines.append(line)

    return result_lines

# 示例使用
log_filename = 'bugreport-goku-UKQ1.240116.001-2024-07-06-22-06-59.txt'
target_time_str = '07-06 22:04:40'
pre_delta_minutes = 3
post_delta_minutes = 2

logs = extract_logs(log_filename, target_time_str, pre_delta_minutes, post_delta_minutes)

# 筛选包含关键词的日志
def filter_logs_by_keywords(logs, keywords):
    filtered_logs = []
    for line in logs:
        if any(keyword in line for keyword in keywords):
            filtered_logs.append(line)
    return filtered_logs


# 读取关键词JSON文件
with open('voicetrigger_keywords.json', 'r', encoding="utf-8") as file:
    keywords_data = json.load(file)
keywords = keywords_data['keywords']


# # 将提取的日志输出到一个新的文件
with open('outputNew.txt', 'w', encoding="utf-8") as output_file:
    output_file.writelines(logs)

# 筛选包含关键词的日志
filtered_logs = filter_logs_by_keywords(logs, keywords)

# 将筛选后的日志输出到一个新的文件
with open('filtered_output.txt', 'w', encoding="utf-8") as output_file:
    output_file.writelines(filtered_logs)



## 优化之后的日志筛选
1. 直接每次分析每2000行的第一条和最后一条日志的时间。如果其中一条日志满足，就直接将这1000行日志最为有效日志存起来；
2. 对于遍历大数量的日志确实能提高效率，比逐行遍历每一条日志提高效率：
    1. 逐行遍历时间复杂度为`O(n)`
    2. 间隔2000行粗略筛选，时间复杂度为`O(n/2000)`
3. 为什么是选择间隔1000行。因为我发现一段有效且连续在一分钟内的日志一定大于1000行。
4. 粗略筛选之后还需要继续细致筛选，因此粗略筛选不用非常详细。


In [20]:
import re
import datetime

# 正则表达式用于匹配时间戳，不包括毫秒部分
timestamp_pattern = re.compile(r'(\d{2}-\d{2} \d{2}:\d{2}:\d{2})')

def parse_log_timestamp(line):
    match = timestamp_pattern.search(line)
    if match:
        timestamp_str = match.group(1)
        return datetime.datetime.strptime(timestamp_str, '%m-%d %H:%M:%S').replace(year=datetime.datetime.now().year)
    return None

def extract_logs(filename, target_time_str, pre_delta_minutes=3, post_delta_minutes=2):
    with open(filename, 'r', encoding="utf-8") as file:
        lines = file.readlines()

    target_time = datetime.datetime.strptime(target_time_str, '%m-%d %H:%M:%S').replace(year=datetime.datetime.now().year)
    # 前3分钟
    pre_delta = datetime.timedelta(minutes=pre_delta_minutes)
    # 后2分钟
    post_delta = datetime.timedelta(minutes=post_delta_minutes)
    
    start_time = target_time - pre_delta
    end_time = target_time + post_delta

    result_lines = []
    step = 2000
    total_lines = len(lines)
    i = 0

    while i < total_lines:
        # 获取当前位置和1000行后的时间戳
        current_time = parse_log_timestamp(lines[i])
        next_i = min(i + step, total_lines - 1)
        next_time = parse_log_timestamp(lines[next_i])

        if (current_time and start_time <= current_time <= end_time) or (next_time and start_time <= next_time <= end_time):
            
            result_lines.extend(lines[i:next_i + 1])
            
        i += step

    return result_lines

# 示例使用
log_filename = 'bugreport-goku-UKQ1.240116.001-2024-07-06-22-06-59.txt'
target_time_str = '07-06 22:04:40'
pre_delta_minutes = 3
post_delta_minutes = 2
logs = extract_logs(log_filename, target_time_str, pre_delta_minutes, post_delta_minutes)


# 筛选包含关键词的日志
def filter_logs_by_keywords(logs, keywords):
    filtered_logs = []
    for line in logs:
        if any(keyword in line for keyword in keywords):
            filtered_logs.append(line)
    return filtered_logs


# 读取关键词JSON文件
with open('voicetrigger_keywords.json', 'r', encoding="utf-8") as file:
    keywords_data = json.load(file)
keywords = keywords_data['keywords']

# 将提取的日志输出到一个新的文件
# with open('output.txt', 'w', encoding="utf-8") as output_file:
#     output_file.writelines(logs)

# 筛选包含关键词的日志
filtered_logs = filter_logs_by_keywords(logs, keywords)

# 将筛选后的日志输出到一个新的文件
with open('new_filtered_output.txt', 'w', encoding="utf-8") as output_file:
    output_file.writelines(filtered_logs)



## 分析规定时间段的一级事件、二级事件

有一个很有意思的地方，只有收到一级唤醒事件，才能得到唤醒方案和相关算法。

### 1. 未收到一级唤醒事件
四种方案通用
未查询到关键日志点‘onRecognition:’，则标记为：未接收到一级事件
```bash
(高通/自研E) D/SVA-LegacyWakeupSession - onRecognition: recognitionEvent = KeyphraseRecognitionEvent [keyphraseExtras=[KeyphraseRecognitionExtra [id=0, recognitionModes=3, coarseConfidenceLevel=98, confidenceLevels=[ConfidenceLevel [userId=1, confidenceLevel=14]]]], status=0, soundModelHandle=1, captureAvailable=true, captureSession=25, captureDelayMs=0, capturePreambleMs=0, triggerInData=false, sampleRate=16000, encoding=2, channelMask=16, data=8944]
(MTK) V/VoiceTriggerManager - onRecognition: GenericRecognitionEvent ::RecognitionEvent [status=0, soundModelHandle=-1, captureAvailable=false, captureSession=-1, captureDelayMs=-1, capturePreambleMs=-1, triggerInData=false, sampleRate=16000, encoding=2, channelMask=1, data=0]
(MTK自研) V/MTKVoiceTriggerManager - onRecognition: GenericRecognitionEvent ::RecognitionEvent [status=0, soundModelHandle=-1, captureAvailable=false, captureSession=297, captureDelayMs=8704, capturePreambleMs=-1, triggerInData=false, sampleRate=16000, encoding=2, channelMask=1, data=0] false
```



### 2. 一级唤醒事件失败

只适用于高通方案
查询到关键日志点‘final_result=false’，则标记为：未通过一级
```bash
final_result=false, first_stage_detect= true, cnn_detect= false, vop_detect=false；---唤醒词、声纹均验证不通过（与柳军确认，cnn/vop检测不分先后，任意一个false，就不会再去验证另一个）
final_result=false, first_stage_detect= true, cnn_detect= false, vop_detect=true；---唤醒词验证不通过
final_result=false, first_stage_detect=true, cnn_detect=true, vop_detect=false；---声纹检测不通过
final_result= true, first_stage_detect=true, cnn_detect= true, vop_detect=true；---验证通过
```


### 3. 非正常一级事件
查询到一次‘onRecognition: recognitionEvent’事件后，发现其采样率‘sampleRate=0’、数据都有问题，则输出：非正常一级事件
```bash
D/SVA-LegacyWakeupSession - onRecognition: recognitionEvent = 
KeyphraseRecognitionEvent [keyphraseExtras=[], status=1, 
soundModelHandle=2, captureAvailable=true, captureSession=0, 
captureDelayMs=0, capturePreambleMs=0, triggerInData=false, 
sampleRate=0, encoding=0, channelMask=0, data=0]
```


### 3. 二级唤醒事件失败
#### 3.1 二级引擎不通过
- 日志定位
**该日志点目前仅自研方案存在**
    - 先定位到：‘PhraseWakeupResult’；
    - 再定位到：‘isVoconWakeupPassed’和‘isVBPassed’
该条日志共四种情况：
1. 若isVoconWakeupPassed=true、isVBPassed=true，则通过二级引擎
```bash
D/MIXWVPCallback - PhraseWakeupResult{mVoconPhrase='小爱同学', 
isVoconWakeupPassed=true, mWakeupStartTime=-1, mWakeupEndTime=-1, 
mScore=23.421255, isAec=false, isVBPassed=true, mWakeupResultStart=1084, 
mWakeupResultEnd=1232}
```

2. 若isVoconWakeupPassed=false、isVBPassed=true，则标记为：未通过二级唤醒引擎
```bash
D/MIXWVPCallback - PhraseWakeupResult{mVoconPhrase='小爱同学', 
isVoconWakeupPassed=false, mWakeupStartTime=-1, mWakeupEndTime=-1, 
mScore=0.0, isAec=false, isVBPassed=true, mWakeupResultStart=0, 
mWakeupResultEnd=0}
```

3. 若isVoconWakeupPassed=true、isVBPassed=false，则标记为：未通过二级声纹引擎
```bash
D/MIXWVPCallback - PhraseWakeupResult{mVoconPhrase='小爱同学', 
isVoconWakeupPassed=true, mWakeupStartTime=-1, mWakeupEndTime=-1, 
mScore=29.038092, isAec=false, isVBPassed=false, mWakeupResultStart=552, 
mWakeupResultEnd=641}
```

4. 若isVoconWakeupPassed=true、isVBPassed=false，则标记为：未通过二级唤醒引擎及声纹引擎
```bash
D/MIXWVPCallback - PhraseWakeupResult{mVoconPhrase='小爱同学', 
isVoconWakeupPassed=false, mWakeupStartTime=-1, mWakeupEndTime=-1, 
mScore=0.0, isAec=false, isVBPassed=false, mWakeupResultStart=0, 
mWakeupResultEnd=0}
```




